# Errors and Exceptions

There are (at least) two distinguishable kinds of errors: syntax errors and exceptions.

## Syntax Errors / Parsing Errors

```python
while True print('Hello world')
  File "<stdin>", line 1
    while True print('Hello world')
               ^^^^^
SyntaxError: invalid syntax
```

The parser repeats the offending line and displays little arrows pointing at the place where the error was detected. Note that this is not always the place that needs to be fixed. In the example, the error is detected at the function print(), since a colon (':') is missing just before it.

The file name (<stdin> in our example) and line number are printed so you know where to look in case the input came from a file.



## Exceptions

Even if a statement or expression is syntactically correct, it may cause an error when an attempt is made to execute it. Errors detected during execution are called exceptions and are not unconditionally fatal: you will soon learn how to handle them in Python programs. Most exceptions are not handled by programs, however, and result in error messages as shown here:

```python
10 * (1/0)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    10 * (1/0)
          ~^~
ZeroDivisionError: division by zero
4 + spam*3
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    4 + spam*3
        ^^^^
NameError: name 'spam' is not defined
'2' + 2
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    '2' + 2
    ~~~~^~~
TypeError: can only concatenate str (not "int") to str
```

The last line of the error message indicates what happened. Exceptions come in different types, and the type is printed as part of the message: the types in the example are ZeroDivisionError, NameError and TypeError. The string printed as the exception type is the name of the built-in exception that occurred. This is true for all built-in exceptions, but need not be true for user-defined exceptions (although it is a useful convention). Standard exception names are built-in identifiers (not reserved keywords).

## Handling Exceptions

It is possible to write programs that handle selected exceptions.

Look at the following example, which asks the user for input until a valid integer has been entered, but allows the user to interrupt the program (using Control-C or whatever the operating system supports); note that a user-generated interruption is signalled by raising the KeyboardInterrupt exception.

In [3]:
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:
        print("Oops!  That was no valid number.  Try again...")

The try statement works as follows.

First, the try clause (the statement(s) between the try and except keywords) is executed.

If no exception occurs, the except clause is skipped and execution of the try statement is finished.

If an exception occurs during execution of the try clause, the rest of the clause is skipped. Then, if its type matches the exception named after the except keyword, the except clause is executed, and then execution continues after the try/except block.

If an exception occurs which does not match the exception named in the except clause, it is passed on to outer try statements; if no handler is found, it is an unhandled exception and execution stops with an error message.

A try statement may have more than one except clause, to specify handlers for different exceptions. At most one handler will be executed. Handlers only handle exceptions that occur in the corresponding try clause, not in other handlers of the same try statement. An except clause may name multiple exceptions as a parenthesized tuple, for example:

```python
... except (RuntimeError, TypeError, NameError):
...     pass
```

In [8]:
def demonstrate_try_except():
    """
    Illustrates the use of multiple except clauses in a try statement.
    Shows how handlers are scoped to the try clause and only one is executed.
    """

    # Try clause 1: Handling a specific exception
    try:
        result = 10 / 0  # This will raise a ZeroDivisionError
        print("This line will not be executed.")
    except ZeroDivisionError:
        print("Caught a ZeroDivisionError in the first try clause.")
    
    # Try clause 2: Handling a different exception
    try:
        # This time, we'll try to convert a string to an integer, which will fail
        num_str = "hello"
        num = int(num_str)
        print("This line will not be executed.")
    except ValueError:
        print("Caught a ValueError in the second try clause.")
    # Try clause 3:  Handles another type of error
    # try:
        # This will cause an AttributeError
    #     my_dict = {"a": 1}
    #     print(my_dict.not_found_key()) # type: ignore # Raises KeyError
    # except KeyError:
    #     print("Caught a KeyError in the third try clause")
    
    print("This line will always be executed.")


if __name__ == "__main__":
    demonstrate_try_except()

Caught a ZeroDivisionError in the first try clause.
Caught a ValueError in the second try clause.
This line will always be executed.


**Explanation and how it relates to the paragraph:**

1. **`try` Clause:** The `try` statement is the core. It contains the code that might raise an exception.  In this example, we have three separate `try` blocks.

2. **`except` Clause:** The `except` clause specifies what to do if a certain type of exception occurs within the corresponding `try` clause.

3. **Multiple `except` Clauses:** This code demonstrates the ability to have multiple `except` clauses. Each `except` clause handles a different type of exception.

4. **One Handler Executed:** The paragraph is crucial here:  *At most one handler will be executed.*  When an exception occurs, only the *first* `except` clause that matches the exception type will be executed.  After that, execution continues *outside* of the `try` statement.

5. **Scoped Handlers:**  The handlers are *scoped* to the `try` statement they're associated with.  If an exception occurs in `try` block 1, it won't affect the execution of `try` block 2 or 3.  Similarly, a `ValueError` handled in `try` block 2 won't affect the handling of `ZeroDivisionError` in `try` block 1.


**Key takeaways from the example:**

*   The `try` statement can contain multiple blocks of code that might raise exceptions.
*   You can use multiple `except` clauses to handle different exception types.
*   Only *one* `except` clause will be executed for any given exception.
*   The scope of each `except` clause is limited to the `try` statement where it's defined.


A class in an except clause matches exceptions which are instances of the class itself or one of its derived classes (but not the other way around — an except clause listing a derived class does not match instances of its base classes). For example, the following code will print B, C, D in that order:

In [4]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")

B
C
D


Note that if the except clauses were reversed (with except B first), it would have printed B, B, B — the first matching except clause is triggered.

When an exception occurs, it may have associated values, also known as the exception’s arguments. The presence and types of the arguments depend on the exception type.

The except clause may specify a variable after the exception name. The variable is bound to the exception instance which typically has an args attribute that stores the arguments. For convenience, builtin exception types define `__str__()` to print all the arguments without explicitly accessing .args.

The exception’s `__str__()` output is printed as the last part (‘detail’) of the message for unhandled exceptions.

In [9]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # the exception type
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
    x, y = inst.args     # unpack args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


**1. Raising the Exception**

   * `raise Exception('spam', 'eggs')`: This line is crucial. It explicitly raises an `Exception` object (which is a base class for all exceptions in Python). It's passing two arguments ('spam' and 'eggs') to the constructor of the `Exception` class. These arguments are bundled together in the exception object, just like we talked about.

**2. Accessing the Exception Object (`inst`)**

   * `except Exception as inst:`:  This is the core of exception handling. The `as inst` part assigns the exception object that was raised to a variable named `inst`.  `inst` is now a reference to the exception instance that's created when the `raise` statement executes.

**3. Printing the Exception Type (`type(inst)`)**

   * `print(type(inst))`: This line shows you the *type* of the exception object. In this case, it will print `<class 'Exception'>`.  This confirms that `inst` contains an instance of the `Exception` base class. It demonstrates that the exception is stored within the `inst` variable.

**4. Accessing Arguments via `.args`**

   * `print(inst.args)`: This accesses the tuple of arguments that were passed to the exception’s constructor.  `inst.args` is a tuple containing ('spam', 'eggs').  This is a direct demonstration of accessing the arguments associated with the exception.

**5. Printing the Exception Object Directly (`print(inst)`)**

   * `print(inst)`: This is where the magic of the `__str__()` method comes in.  Python automatically calls the `__str__()` method on the exception object when you try to print it directly.  The `__str__()` method is designed to provide a user-friendly representation of the exception, and in this case, it automatically prints the arguments stored in `inst.args`.  The output will be: `('spam', 'eggs')`.

   * **Important Note:** This is *not* the same as accessing `inst.args` directly. The `__str__()` method handles the presentation, but `inst.args` is where the arguments are stored internally.

**6. Unpacking Arguments (`x, y = inst.args`)**

   * `x, y = inst.args`: This line demonstrates *unpacking* the arguments from the `inst.args` tuple.  The tuple `('spam', 'eggs')` is unpacked into the variables `x` and `y`.  After this line executes, `x` will be 'spam' and `y` will be 'eggs'.

**In Summary – How it Illustrates the Concepts**

* **Exception Arguments:** This code directly shows that exceptions can carry arguments, which are stored in the `.args` attribute of the exception object.
* **Accessing Arguments:** It demonstrates how to access these arguments using both the `.args` attribute and the `__str__()` method.
* **Unpacking Arguments:**  It showcases how to unpack the arguments from the tuple.
* **Error Information:** The whole code snippet exemplifies how exception arguments provide valuable information about the error that occurred.

**Output of the Code:**

```
<class 'Exception'>
('spam', 'eggs')
x = spam
y = eggs
```


**Understanding Exception Arguments**

When an error (an exception) happens in your Python code – like trying to divide by zero, trying to access an index that’s out of range for a list, or having a file that doesn’t exist – Python raises an exception.  These exceptions aren't just about saying "something went wrong". They often carry *information* about *why* it went wrong. This information is bundled together as "arguments" associated with the exception object.

* **Purpose:** These arguments help you understand the details of the error, such as the value that caused the problem, the filename, or the specific index that was out of bounds.
* **Variety:** The exact arguments depend on the *type* of exception you’re dealing with.  A `ValueError` might have an argument called `args` containing the invalid value, while a `FileNotFoundError` might have `args` containing the name of the missing file.
* **`args` Attribute:**  Most exceptions (especially built-in ones) have an attribute called `args`. This `args` attribute is a *tuple* that stores the arguments passed to the exception's constructor.

**How to Access Exception Arguments**

You can access these arguments in a few ways:

1. **Using the `.args` Attribute:**  This is the standard and most direct way.

2. **Using `__str__()` (for Built-in Exceptions):**  Built-in exceptions (like `ValueError`, `TypeError`, `IndexError`, `FileNotFoundError`) automatically define a `__str__()` method. This method, when called on the exception object, will typically print all the exception’s arguments in a user-friendly way.  You don’t need to explicitly access `args` in this case.

**Python Code Example**

```python
# Example 1: IndexError
try:
    my_list = [1, 2, 3]
    print(my_list[5])  # Attempt to access index 5 (which is out of range)
except IndexError as e:
    print("IndexError occurred!")
    print(f"Arguments: {e.args}")  # Access the arguments via the exception object
    print(f"Argument 0: {e.args[0]}")  # Access the first argument (the index)


# Example 2: ValueError
try:
    num_str = "abc"
    num = int(num_str)  # Attempt to convert "abc" to an integer
    print(num)
except ValueError as e:
    print("ValueError occurred!")
    print(f"Arguments: {e.args}")  # Print all arguments
    #or
    print(f"Invalid value: {e.args[0]}") # Access the invalid value


# Example 3: FileNotFoundError
try:
    with open("nonexistent_file.txt", "r") as f:
        content = f.read()
        print(content)
except FileNotFoundError as e:
    print("FileNotFoundError occurred!")
    print(f"Arguments: {e.args}") # prints the file path
    print(f"Missing file: {e.args[0]}")
```

**Explanation of the Code:**

1. **`try...except` Block:** This structure allows you to handle exceptions gracefully. The code in the `try` block is where you might expect an exception to occur. If an exception is raised, the code in the `except` block is executed.

2. **`as e`:**  This part of the `except` clause assigns the exception object to a variable named `e`.  This variable lets you work with the specific exception that was raised.

3. **`e.args`:** This is the key part.  `e.args` is a tuple that contains all the arguments passed to the exception’s constructor.  You can access these arguments using indexing (e.g., `e.args[0]`).

4. **`__str__()` (Implicit):** When you print the exception object (`print(e)`), Python automatically calls the `__str__()` method on the exception object.  The `__str__()` method prints the information about the exception, including its arguments.

**Key Takeaways:**

* **Error Information:** Exception arguments provide valuable details about the cause of an error.
* **Exception Object:** You work with exceptions through the exception object (`e`).
* **`args` Attribute:** Use the `e.args` attribute to access the arguments.
* **`__str__()` (for Built-ins):**  Leverage the automatic printing provided by the `__str__()` method for convenience, especially when dealing with built-in exceptions.


In [12]:
# Example 1: IndexError
try:
    my_list = [1, 2, 3]
    print(my_list[5])  # Attempt to access index 5 (which is out of range)
except IndexError as e:
    print("IndexError occurred!")
    print(f"Arguments: {e.args}")  # Access the arguments via the exception object
    print(f"Argument 0: {e.args[0]}")  # Access the first argument (the index)


IndexError occurred!
Arguments: ('list index out of range',)
Argument 0: list index out of range


In [16]:
# Example 2: ValueError
try:
    num_str = "abc"
    num = int(num_str)  # Attempt to convert "abc" to an integer
    print(num)
except ValueError as e:
    print("ValueError occurred!")
    print(f"Arguments: {e.args}")  # Print all arguments
    # or
    print(f"Invalid value: {e.args[0]}") # Access the invalid value


ValueError occurred!
Arguments: ("invalid literal for int() with base 10: 'abc'",)
Invalid value: invalid literal for int() with base 10: 'abc'


In [17]:
# Example 3: FileNotFoundError
try:
    with open("nonexistent_file.txt", "r") as f:
        content = f.read()
        print(content)
except FileNotFoundError as e:
    print("FileNotFoundError occurred!")
    print(f"Arguments: {e.args}") # prints the file path
    print(f"Missing file: {e.args[0]}")

FileNotFoundError occurred!
Arguments: (2, 'No such file or directory')
Missing file: 2


BaseException is the common base class of all exceptions. One of its subclasses, Exception, is the base class of all the non-fatal exceptions. Exceptions which are not subclasses of Exception are not typically handled, because they are used to indicate that the program should terminate. They include SystemExit which is raised by sys.exit() and KeyboardInterrupt which is raised when a user wishes to interrupt the program.

Exception can be used as a wildcard that catches (almost) everything. However, it is good practice to be as specific as possible with the types of exceptions that we intend to handle, and to allow any unexpected exceptions to propagate on.

The most common pattern for handling Exception is to print or log the exception and then re-raise it (allowing a caller to handle the exception as well):

In [3]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error:", err)
except ValueError:
    print("Could not convert data to an integer.")
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


The try … except statement has an optional else clause, which, when present, must follow all except clauses. It is useful for code that must be executed if the try clause does not raise an exception. For example:

In [10]:
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except OSError:
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

cannot open --f=c:\Users\ADMIN\AppData\Roaming\jupyter\runtime\kernel-v3580a85a6f3734723f8a459bcb5547907bbbbbf9b.json


**Line-by-Line Explanation:**

1.  `for arg in sys.argv[1:]:`
    *   This line initiates a `for` loop that iterates through the command-line arguments passed to the Python script.
    *   `sys.argv` is a list that contains the command-line arguments.
    *   `sys.argv[0]` is always the name of the script itself.
    *   `sys.argv[1:]` creates a slice of the `sys.argv` list, starting from the second element (index 1) to the end.  This means the loop will process each argument *after* the script name.
    *   In each iteration, the current argument is assigned to the variable `arg`.

2.  `try:`
    *   This starts the `try` block. The code within the `try` block is the code that might potentially raise an exception (an error).  In this case, it attempts to open a file specified by the `arg` variable.

3.  `f = open(arg, 'r')`
    *   This is the core file operation.  It attempts to open the file specified by the `arg` variable in read mode ('r').
    *   If the file exists and is accessible, the `open()` function returns a file object, which is assigned to the variable `f`.
    *   **If the file cannot be opened** (e.g., the file doesn't exist, you don't have permission to read it, or the filename is incorrect), the `open()` function raises an `OSError` (Operating System Error).

4.  `except OSError:`
    *   This is the `except` block. It catches the `OSError` that might be raised by the `open()` function in the `try` block.
    *   If an `OSError` occurs, the code inside this `except` block is executed.
    *   In this case, it prints a message to the console indicating that the file could not be opened, along with the name of the file (`arg`).

5.  `print('cannot open', arg)`
    *   This line prints a user-friendly error message to the console, informing the user that the file could not be opened.

6.  `else:`
    *   This is the crucial part for understanding the concept we're discussing. The `else` clause in a `try...except...else` block is executed *only if* no exception was raised in the `try` block.  In other words, the `else` block is executed only when the `try` block completes successfully (without encountering an error).

7.  `print(arg, 'has', len(f.readlines()), 'lines')`
    *   This line is executed *only if* the `open()` function succeeded.  It prints the name of the file (`arg`) along with the number of lines in the file.
    *   `f.readlines()` reads all the lines from the file into a list of strings.
    *   `len(f.readlines())` calculates the number of elements in the list, which is the number of lines in the file.

8.  `f.close()`
    *   This line closes the file object `f`. It's essential to close files after you're finished with them to release resources and ensure that changes are saved to the file.  It's generally a good practice to always `close()` files you open.

**Explanation of the `else` Clause and How it Works:**

The `else` clause in `try...except...else` provides a distinct block of code that runs when the `try` block completes without any exceptions. This is a useful pattern when you want to perform some action only if an operation succeeded, and you don't want to execute that code if an error occurred.

In this code, the `else` clause allows us to print the number of lines in a file *only* if the file was successfully opened and read. If the file cannot be opened (due to an `OSError`), the `except` block is executed, and the `else` clause is skipped.

**In essence:**

*   **`try`**:  The code you want to execute, which might raise an exception.
*   **`except`**:  The code to handle a specific exception.
*   **`else`**: The code to execute if *no* exception was raised in the `try` block.



The use of the else clause is better than adding additional code to the try clause because it avoids accidentally catching an exception that wasn’t raised by the code being protected by the try … except statement.

Exception handlers do not handle only exceptions that occur immediately in the try clause, but also those that occur inside functions that are called (even indirectly) in the try clause. For example:

In [11]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


## Raising Exceptions

The raise statement allows the programmer to force a specified exception to occur. For example:

In [ ]:
raise NameError('HiThere')
# Traceback (most recent call last):
#   File "<stdin>", line 1, in <module>
#     raise NameError('HiThere')
# NameError: HiThere

The sole argument to raise indicates the exception to be raised. This must be either an exception instance or an exception class (a class that derives from BaseException, such as Exception or one of its subclasses). If an exception class is passed, it will be implicitly instantiated by calling its constructor with no arguments:

In [ ]:
raise ValueError  # shorthand for 'raise ValueError()'

If you need to determine whether an exception was raised but don’t intend to handle it, a simpler form of the raise statement allows you to re-raise the exception:

In [ ]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise

## Exception Chaining

If an unhandled exception occurs inside an except section, it will have the exception being handled attached to it and included in the error message:

In [ ]:
try:
    open("database.sqlite")
except OSError:
    raise RuntimeError("unable to handle error")

# Traceback (most recent call last):
#   File "<stdin>", line 2, in <module>
#     open("database.sqlite")
#     ~~~~^^^^^^^^^^^^^^^^^^^
# FileNotFoundError: [Errno 2] No such file or directory: 'database.sqlite'

# During handling of the above exception, another exception occurred:

# Traceback (most recent call last):
#   File "<stdin>", line 4, in <module>
#     raise RuntimeError("unable to handle error")
# RuntimeError: unable to handle error

To indicate that an exception is a direct consequence of another, the raise statement allows an optional from clause:

In [ ]:
# exc must be exception instance or None.
raise RuntimeError from exc

This can be useful when you are transforming exceptions. For example:


In [ ]:
def func():
    raise ConnectionError

try:
    func()
except ConnectionError as exc:
    raise RuntimeError('Failed to open database') from exc

# Traceback (most recent call last):
#   File "<stdin>", line 2, in <module>
#     func()
#     ~~~~^^
#   File "<stdin>", line 2, in func
# ConnectionError

# The above exception was the direct cause of the following exception:

# Traceback (most recent call last):
#   File "<stdin>", line 4, in <module>
#     raise RuntimeError('Failed to open database') from exc
# RuntimeError: Failed to open database

It also allows disabling automatic exception chaining using the from None idiom:

In [ ]:
try:
    open('database.sqlite')
except OSError:
    raise RuntimeError from None

## User-defined Exceptions

Programs may name their own exceptions by creating a new exception class (see Classes for more about Python classes). Exceptions should typically be derived from the Exception class, either directly or indirectly.

Exception classes can be defined which do anything any other class can do, but are usually kept simple, often only offering a number of attributes that allow information about the error to be extracted by handlers for the exception.

Most exceptions are defined with names that end in “Error”, similar to the naming of the standard exceptions.

Many standard modules define their own exceptions to report errors that may occur in functions they define.

## Defining Clean-up Actions

The try statement has another optional clause which is intended to define clean-up actions that must be executed under all circumstances. For example:

In [12]:
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

A `finally` block is a section of code that *always* runs, no matter what happens inside the `try` block. Think of it as a cleanup routine. It guarantees that certain actions will happen, even if there’s an error (an exception) that needs fixing.

**Here’s how it plays out:**

1. **Error Happens:** If an error (exception) pops up while the code in the `try` block is running, the program first looks for an `except` block that can handle it.

2. **Handling the Error:**  
   * **If an `except` block catches the error:** The code in that `except` block runs, and the error is handled. The `finally` block *still* runs afterwards.
   * **If no `except` block handles the error:** The error isn't caught. The program then executes the `finally` block *before* letting the error bubble up and potentially crash the program.

3. **`else` Blocks:**  If you have an `else` block, it runs *only* if the `try` block executes successfully (i.e., no exceptions). It doesn't run if an exception occurs.

4. **Control Flow:**  
   * **`break`, `continue`, `return` in `finally`:** If the `finally` block itself contains a `break`, `continue`, or `return` statement, the program *doesn’t* re-raise the original exception. It just exits from the `try` block (or loop) using the value from the `finally` block.
   * **`break`, `continue`, `return` in `try`:** If the `try` block has a `break`, `continue` or `return`, *before* it executes, the `finally` block *will* run.

5. **`return` Statements:** If a `finally` block has a `return` statement, it’s important to know that the value returned by the `finally` block’s `return` is the value that gets used, not the value returned from the `try` block’s `return` statement.


**In short:** `finally` is for cleanup, and it *always* runs, regardless of whether an exception happens or not.  It’s a crucial part of writing robust code!


In [13]:
def bool_return():
    try:
        return True
    finally:
        return False

bool_return()

False

In [ ]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

divide(2, 1)


divide(2, 0)


divide("2", "1")

result is 2.0
executing finally clause
division by zero!
executing finally clause
type error
executing finally clause


As you can see, the finally clause is executed in any event. The TypeError raised by dividing two strings is not handled by the except clause and therefore re-raised after the finally clause has been executed.

In real world applications, the finally clause is useful for releasing external resources (such as files or network connections), regardless of whether the use of the resource was successful.

## Predefined Cean-up Actions

Some objects define standard clean-up actions to be undertaken when the object is no longer needed, regardless of whether or not the operation using the object succeeded or failed. Look at the following example, which tries to open a file and print its contents to the screen.

In [ ]:
for line in open("Exceptions_summary.md"):
    print(line, end="")

The problem with this code is that it leaves the file open for an indeterminate amount of time after this part of the code has finished executing. This is not an issue in simple scripts, but can be a problem for larger applications. The with statement allows objects like files to be used in a way that ensures they are always cleaned up promptly and correctly.

In [ ]:
with open("Exceptions_summary.md") as f:
    for line in f:
        print(line, end="")

After the statement is executed, the file f is always closed, even if a problem was encountered while processing the lines. Objects which, like files, provide predefined clean-up actions will indicate this in their documentation.

## Raising and Handling Multiple Unrelated Exceptions

There are situations where it is necessary to report several exceptions that have occurred. This is often the case in concurrency frameworks, when several tasks may have failed in parallel, but there are also other use cases where it is desirable to continue execution and collect multiple errors rather than raise the first exception.

The builtin ExceptionGroup wraps a list of exception instances so that they can be raised together. It is an exception itself, so it can be caught like any other exception.

In [ ]:
def f():
    excs = [OSError('error 1'), SystemError('error 2')]
    raise ExceptionGroup('there were problems', excs)

f()

try:
    f()
except Exception as e:
    print(f'caught {type(e)}: e')

  + Exception Group Traceback (most recent call last):
  |   File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |     ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\770729662.py", line 5, in <module>
  |     f()
  |     ~^^
  |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\770729662.py", line 3, in f
  |     raise ExceptionGroup('there were problems', excs)
  | ExceptionGroup: there were problems (2 sub-exceptions)
  +-+---------------- 1 ----------------
    | OSError: error 1
    +---------------- 2 ----------------
    | SystemError: error 2
    +------------------------------------


By using except* instead of except, we can selectively handle only the exceptions in the group that match a certain type. In the following example, which shows a nested exception group, each except* clause extracts from the group exceptions of a certain type while letting all other exceptions propagate to other clauses and eventually to be reraised.

In [24]:
def f():
    raise ExceptionGroup(
        "group1",
        [
            OSError(1),
            SystemError(2),
            ExceptionGroup(
                "group2",
                [
                    OSError(3),
                    RecursionError(4)
                ]
            )
        ]
    )

try:
    f()
except* OSError as e:
    print("There were OSErrors")
except* SystemError as e:
    print("There were SystemErrors")

There were OSErrors
There were SystemErrors


  + Exception Group Traceback (most recent call last):
  |   File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |     ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\1308333313.py", line 18, in <module>
  |     f()
  |     ~^^
  |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\1308333313.py", line 2, in f
  |     raise ExceptionGroup(
  |     ...<12 lines>...
  |     )
  | ExceptionGroup: group1 (1 sub-exception)
  +-+---------------- 1 ----------------
    | ExceptionGroup: group2 (1 sub-exception)
    +-+---------------- 1 ----------------
      | RecursionError: 4
      +------------------------------------


Note that the exceptions nested in an exception group must be instances, not types. This is because in practice the exceptions would typically be ones that have already been raised and caught by the program, along the following pattern:

In [ ]:
excs = []
for test in tests:
    try:
        test.run()
    except Exception as e:
        excs.append(e)

if excs:
   raise ExceptionGroup("Test Failures", excs)

## Enriching Exceptions with Notes

When an exception is created in order to be raised, it is usually initialized with information that describes the error that has occurred. There are cases where it is useful to add information after the exception was caught. For this purpose, exceptions have a method add_note(note) that accepts a string and adds it to the exception’s notes list. The standard traceback rendering includes all notes, in the order they were added, after the exception.

In [26]:
try:
    raise TypeError('bad type')
except Exception as e:
    e.add_note('Add some information')
    e.add_note('Add some more information')
    raise

TypeError: bad type

For example, when collecting exceptions into an exception group, we may want to add context information for the individual errors. In the following each exception in the group has a note indicating when this error has occurred.

In [27]:
def f():
    raise OSError('operation failed')

excs = []
for i in range(3):
    try:
        f()
    except Exception as e:
        e.add_note(f'Happened in Iteration {i+1}')
        excs.append(e)

raise ExceptionGroup('We have some problems', excs)

  + Exception Group Traceback (most recent call last):
  |   File "C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |     ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\1429863547.py", line 12, in <module>
  |     raise ExceptionGroup('We have some problems', excs)
  | ExceptionGroup: We have some problems (3 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\1429863547.py", line 7, in <module>
    |     f()
    |     ~^^
    |   File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_8724\1429863547.py", line 2, in f
    |     raise OSError('operation failed')
    | OSError: operation failed
    | Happened in Iterat